In [1]:
#import torch
from transformers import BertTokenizer
#from bert_rnn_depression_model import BertRNNClassifier 

import torch
import torch.nn as nn
from transformers import BertModel

# Define the BERT + RNN model
class BertRNNClassifier(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", hidden_dim=256, num_classes=2):
        super(BertRNNClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.rnn = nn.RNN(input_size=768, hidden_size=hidden_dim, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        rnn_out, _ = self.rnn(bert_output.last_hidden_state)
        rnn_out = rnn_out[:, -1, :]  # Get the last RNN output
        output = self.fc(self.dropout(rnn_out))
        return self.softmax(output)
        
model = BertRNNClassifier()
model_path = "C:/Users/priya/OneDrive/Desktop/Depression detection/bert_rnn_depression_model.pth" 
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load("bert_rnn_depression_model.pth", map_location=device))
model.eval()  # Set model to evaluation mode

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to predict depression status
def predict(text):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids, attention_mask = inputs["input_ids"].to(device), inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        prediction = torch.argmax(outputs, dim=1).item()

    return "Depressed" if prediction == 1 else "Not Depressed"

print("Ready to predict.")


Ready to predict.


In [3]:
text_input = input("Enter a text to predict : ")
prediction = predict(text_input)
print(f"Prediction: {prediction}")

Enter a text to predict :  nothing look forward lifei dont many reasons keep going feel like nothing keeps going next day makes want hang myself


Prediction: Depressed
